## 訓練 Training

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D, BatchNormalization
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
# 訓練樣本目錄和測試樣本目錄
train_dir = '/content/drive/MyDrive/HW4/train'
test_dir = '/content/drive/MyDrive/HW4/validation'

# 對訓練圖像做正規化
train_pic_gen = ImageDataGenerator(rescale=1./255)

# 對測試圖像做正規化
test_pic_gen = ImageDataGenerator(rescale=1./255)

In [4]:
# 利用 .flow_from_directory 函數生成訓練數據
train_flow = train_pic_gen.flow_from_directory(train_dir,
                        target_size=(224,224),
                        batch_size=64,
                        class_mode='categorical')

# 利用 .flow_from_directory 函數生成測試數據
test_flow = test_pic_gen.flow_from_directory(test_dir,
                      target_size=(224,224),
                      batch_size=64,
                      class_mode='categorical')
print(train_flow.class_indices)

Found 472 images belonging to 2 classes.
Found 116 images belonging to 2 classes.
{'indoor': 0, 'outdoor': 1}


In [5]:
#搭建網路
resize = 224
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs/1',
                      histogram_freq= 0,
                      write_graph=True,
                      write_images=True)

model = Sequential()

# level1
model.add(Conv2D(filters=96,kernel_size=(11,11),
        strides=(4,4),padding='valid',
        input_shape=(resize,resize,3),
        activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),
            strides=(2,2),
            padding='valid'))

# level_2
model.add(Conv2D(filters=256,kernel_size=(5,5),
        strides=(1,1),padding='same',
        activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),
          strides=(2,2),
          padding='valid'))

# layer_3
model.add(Conv2D(filters=384,kernel_size=(3,3),
        strides=(1,1),padding='same',
        activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=384,kernel_size=(3,3),
        strides=(1,1),padding='same',
        activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=356,kernel_size=(3,3),
        activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),
          strides=(2,2),padding='valid'))

# layer_4
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))

# output layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
      optimizer='sgd',
      metrics=['accuracy'])

In [6]:
#train
his = model.fit(train_flow,
        epochs=15,
        verbose=1,
        validation_data=test_flow,
        callbacks=[tbCallBack])

model.save('/content/drive/MyDrive/HW4/my_model.h5')

Epoch 1/15
8/8 [==============================] - 186s 23s/step - loss: 5.6046 - accuracy: 0.4964 - val_loss: 0.7206 - val_accuracy: 0.5086
Epoch 2/15
8/8 [==============================] - 67s 9s/step - loss: 1.3211 - accuracy: 0.5480 - val_loss: 0.6848 - val_accuracy: 0.5345
Epoch 3/15
8/8 [==============================] - 68s 9s/step - loss: 0.8739 - accuracy: 0.6356 - val_loss: 0.6601 - val_accuracy: 0.6121
Epoch 4/15
8/8 [==============================] - 67s 9s/step - loss: 0.6777 - accuracy: 0.6758 - val_loss: 0.6528 - val_accuracy: 0.5690
Epoch 5/15
8/8 [==============================] - 67s 8s/step - loss: 0.5153 - accuracy: 0.7621 - val_loss: 0.6581 - val_accuracy: 0.5431
Epoch 6/15
8/8 [==============================] - 67s 8s/step - loss: 0.5352 - accuracy: 0.7875 - val_loss: 0.6370 - val_accuracy: 0.6293
Epoch 7/15
8/8 [==============================] - 67s 9s/step - loss: 0.5030 - accuracy: 0.7658 - val_loss: 0.6440 - val_accuracy: 0.5776
Epoch 8/15
8/8 [================

## 預測 Testing

In [14]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [9]:
## 載入模型
model = tf.keras.models.load_model('/content/drive/MyDrive/HW4/my_model.h5')

In [15]:
def predict(img):
  test_img = cv2.resize(img,(224,224))
  test_img = np.asarray(test_img.astype('float32'))
  test_img = test_img/255.
  test_img = test_img.reshape((1,224,224,3))

  prediction = model.predict(test_img)
  index = np.argmax(prediction)
  return dictionary[index]
	
dictionary = {0:'indoor', 1:'outdoor'}
for filename in os.listdir('/content/drive/MyDrive/HW4/test'):
  img = cv2.imread('/content/drive/MyDrive/HW4/test/'+filename)
  ans = predict(img)
  print('圖片:{0}, 預測:{1}'.format(filename, ans))

圖片:7109056015_1_indoor.jpg, 預測:indoor
圖片:7109056015_4_indoor.jpg, 預測:indoor
圖片:7109056015_5_indoor.jpg, 預測:indoor
圖片:7109056015_3_indoor.jpg, 預測:indoor
圖片:7109056015_2_indoor.jpg, 預測:indoor
圖片:7109056015_5.jpg, 預測:indoor
圖片:7109056015_3.jpg, 預測:indoor
圖片:7109056015_1.jpg, 預測:indoor
圖片:7109056015_2.jpg, 預測:outdoor
圖片:7109056015_4.jpg, 預測:indoor
